# Kaggle Study 12일차 - 1 (home_credit)

코드출처 : https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm

-  Allstate Faron의 스크립트를 기반으로 Starter를 만듦.
  
링크 : https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867을 참조하십시오.  
  
- ogrellier의 사전 처리를 참고함
  
링크 : https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm을 참조하십시오

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

In [2]:
NFOLDS = 3
SEED = 0
NROWS = None

In [3]:
data = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/home_credit/application_train.csv')
test = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/home_credit/application_test.csv')
prev = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/home_credit/previous_application.csv')

In [4]:
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

In [5]:
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])
    
gc.enable()

y_train = data['TARGET']
del data['TARGET']

In [6]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

In [7]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [8]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

In [9]:
x_train = x_train.fillna(0)
x_test= x_test.fillna(0)

In [10]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [11]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in  excluded_feats]

In [12]:
x_train = x_train[features]
x_test = x_test[features]

In [13]:
kf = KFold(n_splits = NFOLDS, shuffle = True, random_state = SEED)

In [14]:
class SklearnWrapper(object):
    def __init__(self,clf,seed=0,params=None):
        params['random_state']=seed
        self.clf = clf(**params)
        
    def train(self,x_train,y_train):
        self.clf.fit(x_train,y_train)
        
    def predict(self,x):
        return self.clf.predict_proba(x)[:,1]

In [15]:
class CatboostWrapper(object):
    def __init__(self,clf,seed=0,params=None):
        params['random_seed']=seed
        self.clf = clf(**params)
        
    def train(self,x_train,y_train):
        self.clf.fit(x_train,y_train)
        
    def predict(self,x):
        return self.clf.predict_proba(x)[:,1]

In [16]:
class LightGBMWrapper(object):
    def __init__(self,clf,seed=0,params=None):
        params['feature_fraction_seed']=seed
        params['bagging_seed']=seed
        self.clf = clf(**params)
        
    def train(self,x_train,y_train):
        self.clf.fit(x_train,y_train)
        
    def predict(self,x):
        return self.clf.predict_proba(x)[:,1]

In [17]:
class XgbWrapper(object):
    def __init__(self,seed=0,params=None):
        self.param = params
        self.param['seed']=seed
        self.nrounds = params.pop('nrounds',250)
        
    def train(self,x_train,y_train):
        dtrain = xgb.DMatrix(x_train,label=y_train)
        self.gbdt= xgb.train(self.param,dtrain,self.nrounds)
        
    def predict(self,x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [18]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [19]:
et_params = {'n_jobs': 16,'n_estimators': 200,'max_features': 0.5,
             'max_depth': 12,'min_samples_leaf': 2}

rf_params = {'n_jobs': 16,'n_estimators': 200,'max_features': 0.2,
             'max_depth': 12,'min_samples_leaf': 2}

xgb_params = {'seed': 0,'colsample_bytree': 0.7,'silent': 1,'subsample': 0.7,\
              'learning_rate': 0.075,'objective': 'binary:logistic','max_depth': 4,
              'num_parallel_tree': 1,'min_child_weight': 1,'nrounds': 200}

catboost_params = {'iterations': 200,'learning_rate': 0.5,'depth': 3,
                   'l2_leaf_reg': 40,'bootstrap_type': 'Bernoulli','subsample': 0.7,
                   'scale_pos_weight': 5,'eval_metric': 'AUC','od_type': 'Iter',
                   'allow_writing_files': False}

lightgbm_params = {'n_estimators':200,'learning_rate':0.1,'num_leaves':123,
                   'colsample_bytree':0.8,'subsample':0.9,'max_depth':15,
                   'reg_alpha':0.1,'reg_lambda':0.1,'min_split_gain':0.01,
                   'min_child_weight':2}

In [22]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed=SEED, params=lightgbm_params)

In [23]:
xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

0:	total: 118ms	remaining: 23.4s
1:	total: 159ms	remaining: 15.8s
2:	total: 191ms	remaining: 12.6s
3:	total: 229ms	remaining: 11.2s
4:	total: 266ms	remaining: 10.4s
5:	total: 300ms	remaining: 9.7s
6:	total: 336ms	remaining: 9.27s
7:	total: 370ms	remaining: 8.89s
8:	total: 403ms	remaining: 8.55s
9:	total: 438ms	remaining: 8.33s
10:	total: 471ms	remaining: 8.1s
11:	total: 508ms	remaining: 7.96s
12:	total: 541ms	remaining: 7.79s
13:	total: 577ms	remaining: 7.67s
14:	total: 615ms	remaining: 7.58s
15:	total: 655ms	remaining: 7.53s
16:	total: 692ms	remaining: 7.44s
17:	total: 726ms	remaining: 7.34s
18:	total: 761ms	remaining: 7.25s
19:	total: 796ms	remaining: 7.16s
20:	total: 830ms	remaining: 7.07s
21:	total: 868ms	remaining: 7.02s
22:	total: 905ms	remaining: 6.96s
23:	total: 946ms	remaining: 6.94s
24:	total: 985ms	remaining: 6.89s
25:	total: 1.02s	remaining: 6.82s
26:	total: 1.05s	remaining: 6.75s
27:	total: 1.08s	remaining: 6.67s
28:	total: 1.12s	remaining: 6.62s
29:	total: 1.16s	remaining

41:	total: 1.6s	remaining: 6.03s
42:	total: 1.64s	remaining: 5.99s
43:	total: 1.68s	remaining: 5.94s
44:	total: 1.71s	remaining: 5.9s
45:	total: 1.75s	remaining: 5.86s
46:	total: 1.78s	remaining: 5.81s
47:	total: 1.82s	remaining: 5.78s
48:	total: 1.86s	remaining: 5.75s
49:	total: 1.9s	remaining: 5.71s
50:	total: 1.94s	remaining: 5.68s
51:	total: 1.97s	remaining: 5.62s
52:	total: 2.01s	remaining: 5.58s
53:	total: 2.05s	remaining: 5.53s
54:	total: 2.09s	remaining: 5.5s
55:	total: 2.13s	remaining: 5.47s
56:	total: 2.16s	remaining: 5.43s
57:	total: 2.2s	remaining: 5.39s
58:	total: 2.24s	remaining: 5.36s
59:	total: 2.29s	remaining: 5.34s
60:	total: 2.33s	remaining: 5.32s
61:	total: 2.37s	remaining: 5.28s
62:	total: 2.42s	remaining: 5.26s
63:	total: 2.46s	remaining: 5.23s
64:	total: 2.5s	remaining: 5.2s
65:	total: 2.54s	remaining: 5.16s
66:	total: 2.57s	remaining: 5.11s
67:	total: 2.61s	remaining: 5.08s
68:	total: 2.65s	remaining: 5.04s
69:	total: 2.69s	remaining: 4.99s
70:	total: 2.73s	rema

82:	total: 3.23s	remaining: 4.56s
83:	total: 3.27s	remaining: 4.51s
84:	total: 3.31s	remaining: 4.47s
85:	total: 3.34s	remaining: 4.43s
86:	total: 3.38s	remaining: 4.39s
87:	total: 3.42s	remaining: 4.35s
88:	total: 3.45s	remaining: 4.31s
89:	total: 3.49s	remaining: 4.27s
90:	total: 3.53s	remaining: 4.23s
91:	total: 3.57s	remaining: 4.19s
92:	total: 3.6s	remaining: 4.15s
93:	total: 3.64s	remaining: 4.1s
94:	total: 3.68s	remaining: 4.07s
95:	total: 3.72s	remaining: 4.03s
96:	total: 3.75s	remaining: 3.98s
97:	total: 3.79s	remaining: 3.94s
98:	total: 3.83s	remaining: 3.91s
99:	total: 3.87s	remaining: 3.87s
100:	total: 3.91s	remaining: 3.83s
101:	total: 3.95s	remaining: 3.79s
102:	total: 3.99s	remaining: 3.75s
103:	total: 4.02s	remaining: 3.71s
104:	total: 4.05s	remaining: 3.67s
105:	total: 4.09s	remaining: 3.63s
106:	total: 4.13s	remaining: 3.59s
107:	total: 4.17s	remaining: 3.55s
108:	total: 4.21s	remaining: 3.51s
109:	total: 4.24s	remaining: 3.47s
110:	total: 4.28s	remaining: 3.44s
111:	

In [24]:
print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

XG-CV: 0.2594620936343568
ET-CV: 0.26296502347137407
RF-CV: 0.2629856213006646
RF-CV: 0.33114480486162


In [25]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

(307511, 4),(48744, 4)


In [26]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]

test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')